In [1]:
from cpe_dict_loader import *
from registry_query import *
from cpe_similarity import *
from ml_importance import *
import json,random

In [ ]:
software_inventory = get_installed_software()
cpe_list = load_cpe_processed_json()

In [ ]:
test_set = [{"Vendor":"Google Corporation LTD","Name":"Chrome 0.1.38","Version":"1.54345.123"},
            {"Vendor":"Random Corp","Name":"Discord Chat and Message 0.0.4","Version":"0.5"},
            {"Vendor":"Microsoft Corporation","Name":"Microsoft Edge","Version":"80.0"},
            {"Vendor":"Spotify","Name":"Spotify for Windows","Version":"2"},
            {"Vendor":"Python Software Foundation","Name":"Python 2.7 (x64) Binary","Version":"2"}]

with open("test_cpe_set.json","r") as file:
    test_set = json.load(file)

In [ ]:
#******************ENSEMBLE MATCHING******************

all_matches = []

for software in test_set:
    best_match = {"CPE":None,"Similarity":0}

    if software.get("Vendor") and software.get("Name"):  
        name_importance = return_importance_vector(software.get("Name"))
        vendor_importance = return_importance_vector(software.get("Vendor"))

    else:
        continue

    for cpe in cpe_list:         
        score = ensemble_similarity(cpe[1],software.get("Vendor"),software.get("Name"),software.get("Version"),vendor_importance,name_importance)

        if score > best_match["Similarity"] and cpe[1][2] == "a" and cpe[1][10] in ["windows","*"]:
            best_match["CPE"] = cpe[0]
            best_match["Similarity"] = score

    all_matches.append({"Software":software,"Best_match":best_match})

    if all_matches[-1]["Best_match"]["Similarity"] > 0.0: print(all_matches[-1])

In [13]:
create_cpe_txt_cleaned()

In [14]:
import fasttext
model = fasttext.train_unsupervised("NVD_cpes_cleaned.txt")


In [18]:
model.get_nearest_neighbors("microsoft")
#could use this function on the cpe software vendor, and if it returns something similar to what's in the software name or vendor increase the score?
#or better, use it to search for a wider array of cpes: since the names are often different, using this function could give an idea of similar names that are already in the cpe dataset

[(0.9294102787971497, 'microsoft_hyper-v'),
 (0.920555591583252, 'microsoft_azure'),
 (0.8652738928794861, 'microsoft.aspnetcore.mvc'),
 (0.8583853840827942, 'qibosoft'),
 (0.8485546708106995, 'microco'),
 (0.8438853025436401, 'microsoft.aspnetcore.mvc.razor'),
 (0.8423499464988708, 'microk8s'),
 (0.8412947058677673, 'microsoft_exchange'),
 (0.8385860323905945, 'novosoft'),
 (0.8376108407974243, 'microseven')]